# See the diff of (h-bond or non-h-bond) b/w strong and week attention by each head

- hbond combination / Attention strong combination.
- hbond combination / all combination.

In [1]:
import os
import subprocess
import pickle
from tqdm import tqdm
import pandas as pd
from analysis_util import calc_melt_df
from matplotlib import pyplot as plt
from numpy.polynomial.polynomial import polyfit
import matplotlib.pyplot as plt

In [2]:
# %%writefile -a analysis_util.py
def calc_melt_df(distmat_vis, attn_output_weights1, attn_output_weights2):
    separator = [c for c in attn_output_weights2.columns if ':' in c][0]
    separator_pos = attn_output_weights2.columns.get_loc(separator)
    removers = [':', 'C_0'] + \
        [attn_output_weights2.columns[i] for i in [separator_pos, separator_pos-1, separator_pos+1, len(attn_output_weights2.columns)-1]]
    remove_col = [c for c in attn_output_weights2.columns if c in removers]
    remove_ind = [c for c in attn_output_weights1.index if c in removers]
    attn_output_weights2 = attn_output_weights2.drop(columns=remove_col)
    attn_output_weights1 = attn_output_weights1.drop(index=remove_ind)
    
    attn_output_weights2_vis = attn_output_weights2.melt(ignore_index=False).reset_index().rename(columns={'variable':'tcr', 'index':'peptide'})
    attn_output_weights2_vis= attn_output_weights2_vis.sort_values(by=['peptide', 'tcr'])
    attn_output_weights1_vis = attn_output_weights1.T.melt(ignore_index=False).reset_index().rename(columns={'variable':'tcr', 'index':'peptide'})
    attn_output_weights1_vis = attn_output_weights1_vis.sort_values(by=['peptide', 'tcr'])
    
    merged_xy_1 = pd.merge(distmat_vis, attn_output_weights1_vis, on=['peptide','tcr'], how='inner')
    merged_xy_2 = pd.merge(distmat_vis, attn_output_weights2_vis, on=['peptide','tcr'], how='inner')
    return merged_xy_1, merged_xy_2



In [3]:
%%time

HEAD_COUNT=4
STD_THH = 5.0
# DATETIME = '2021'

pickleload = lambda p: pickle.load(open(p,"rb"))

DATETIME = '20220211_191954'

DICT_PDBID_2_Atten12 = pickleload(
    f"/Users/kyoheikoyama/workspace/tcrpred/analysis/DICT_PDB_Result/{DATETIME}_DICT_PDBID_2_Atten12.pickle"
)
DICT_PDBID_2_MELTDIST = pickleload(
    f"/Users/kyoheikoyama/workspace/tcrpred/analysis/DICT_PDB_Result/{DATETIME}_DICT_PDBID_2_MELTDIST.pickle"
)
DICT_PDBID_2_model_out = pickleload(
    f"/Users/kyoheikoyama/workspace/tcrpred/analysis/DICT_PDB_Result/{DATETIME}_DICT_PDBID_2_model_out.pickle"
)

DICT_PDBID_2_CHAINNAMES = pickleload(
    f"/Users/kyoheikoyama/workspace/tcrpred/analysis/DICT_PDB_Result/{DATETIME}_DICT_PDBID_2_CHAINNAMES.pickle"
)

DICT_PDBID_2_CDRS = pickleload(
    f"/Users/kyoheikoyama/workspace/tcrpred/analysis/DICT_PDB_Result/{DATETIME}_DICT_PDBID_2_CDRS.pickle"
)

DICT_PDBID_2_RESIDUES = pickleload(
    f"/Users/kyoheikoyama/workspace/tcrpred/analysis/DICT_PDB_Result/{DATETIME}_DICT_PDBID_2_RESIDUES.pickle"
)

CPU times: user 23.9 s, sys: 1.98 s, total: 25.8 s
Wall time: 26.7 s


# Get hhb and nnb

In [4]:
DIR_LIGPLOTTMP = "/Users/kyoheikoyama/workspace/ligplottmp"
HHB_COLUMNS = [
    'donor_res_code', 'donor_chain_identifier','donor_res_seq_num', 'donor_res_atom_name',
    'acceptor_res_code', 'acceptor_chain_identifier','acceptor_res_seq_num', 'acceptor_res_atom_name',
    'hydrogen_bond_distance', 'plus_minus'
]
ABP_2_NUMBER = {'alpha':0, 'beta':1, 'peptide':2}

In [5]:
for f in os.listdir(DIR_LIGPLOTTMP):
    if len(f)!=7:
        to_delete_dir = os.path.join(DIR_LIGPLOTTMP,f)
        print(to_delete_dir)

/Users/kyoheikoyama/workspace/ligplottmp/hbadd_hbplus_result


In [6]:
# from pdb_util import AACODES_DICT

AACODES = pd.read_csv('./aa_codes.csv')
AACODES_DICT = {row['Abbreviation']:row['1 letter abbreviation'] for i, row in AACODES.iterrows()}
AACODES_DICT_upper = {k.upper():v for k,v in AACODES_DICT.items()}

def cleanup(s): 
    s= s.replace('  ',' ').replace(' ',',').replace(',,',',').replace(',,',',')
    return pd.Series(s.split(','))

def read_hhb(fname):
    df = pd.read_table(fname, skiprows=2)
    df = df.iloc[:,0].apply(cleanup)
    df.columns = HHB_COLUMNS[:len(df.columns)]
    df['AA_code_donor'] = df['donor_res_code'].map(AACODES_DICT_upper)
    df['AA_code_acceptor'] = df['acceptor_res_code'].map(AACODES_DICT_upper)
    return df

def get_df(pdbid):
    f = f'pdb{pdbid.lower()}'
    fname = os.path.join(DIR_LIGPLOTTMP, f, "ligplot.hhb")
    return read_hhb(fname)


def get_pos(pdbid, num, row, tseqnum='donor_res_seq_num', dict_pdbid_2_cdrresidues=DICT_PDBID_2_CDRS):
    if int(row[tseqnum]) in [c.get_id()[1] for c in dict_pdbid_2_cdrresidues[pdbid][num]]:
        pos = [int(c.get_id()[1]) for c in dict_pdbid_2_cdrresidues[pdbid][num]].index(int(row[tseqnum]))
        if num==1: # beta needs another count for pos
            return pos + 1 + len(dict_pdbid_2_cdrresidues[pdbid][0])
        else:
            return pos
    else:
        return None


def add_colname(df_hhb, pdbid, dict_pdbid_2_cdrresidues):
    alpha_len = len(dict_pdbid_2_cdrresidues[pdbid][0])
    colname_donor = []
    for i, row in df_hhb.iterrows():
        donor_num = ABP_2_NUMBER[row['abp_donor']]
        donor_pos = get_pos(pdbid, donor_num, row, 'donor_res_seq_num', dict_pdbid_2_cdrresidues)
        if donor_pos is None: # the h-bond region is out of scope. (Not in CDR.)
            colname_donor.append('')
        else:
            donor_name = row['AA_code_donor'] + '_' + str(donor_pos)
            colname_donor.append(donor_name)

    colname_acceptor = []
    for i, row in df_hhb.iterrows():
        acceptor_num = ABP_2_NUMBER[row['abp_acceptor']]
        acceptor_pos = get_pos(pdbid, acceptor_num, row, 'acceptor_res_seq_num', dict_pdbid_2_cdrresidues)
        if acceptor_pos is None: # the h-bond region is out of scope. (Not in CDR.)
            colname_acceptor.append(None)
        else:
            acceptor_name = row['AA_code_acceptor'] + '_' + str(acceptor_pos)
            colname_acceptor.append(acceptor_name)

    df_hhb['colname_donor'] = colname_donor
    df_hhb['colname_acceptor'] = colname_acceptor    
    return df_hhb

In [7]:
os.listdir(DIR_LIGPLOTTMP)

['pdb1mi5',
 'pdb3vxu',
 'pdb3vxr',
 'pdb2j8u',
 'pdb3kps',
 'pdb2vlk',
 'pdb3mv8',
 'pdb3vxs',
 'pdb3pqy',
 'pdb5men',
 'pdb3mv9',
 'pdb2vlj',
 'pdb3kpr',
 'pdb3mv7',
 'pdb5jzi',
 'pdb3mbe',
 'pdb5nht',
 'pdb6eqb',
 'pdb4jrx',
 'pdb6rpb',
 'pdb2bnq',
 'pdb2z31',
 'pdb4jry',
 'hbadd_hbplus_result',
 'pdb5isz',
 'pdb3d3v',
 'pdb5e9d',
 'pdb4z7v',
 'pdb5nqk',
 'pdb5brz',
 'pdb2uwe',
 'pdb4p4k',
 'pdb4p2r',
 'pdb4p2o',
 'pdb5d2l',
 'pdb3qiu',
 'pdb1u3h',
 'pdb3pwp',
 'pdb2nx5',
 'pdb1kj2',
 'pdb1qrn',
 'pdb3vxm',
 'pdb4qok',
 'pdb2ypl',
 'pdb1fyt',
 'pdb2vlr',
 'pdb1d9k',
 'pdb6q3s',
 'pdb1lp9',
 'pdb5wlg',
 'pdb6avg',
 'pdb4mji',
 'pdb5jhd',
 'pdb4qrr',
 'pdb3w0w',
 'pdb6avf',
 'pdb3utt',
 'pdb2bnr',
 'pdb3uts',
 'pdb6rpa',
 'pdb6eqa',
 'pdb5ks9',
 'pdb3d39',
 'pdb5wkf',
 'pdb4gg6',
 'pdb1ymm',
 'pdb5wkh',
 'pdb4p2q',
 'pdb4ozh',
 'pdb4jfd',
 'pdb4ozf',
 'pdb5tez',
 'pdb4z7u',
 'pdb4ozg',
 'pdb4ozi',
 'pdb4jfe',
 'pdb2icw',
 'pdb1j8h',
 'pdb2ol3']

In [8]:
DICT_PDBID_2_dfhhb = {}

if os.path.exists(f'./DICT_PDB_Result/{DATETIME}_DICT_PDBID_2_dfhhb.pickle'):
    DICT_PDBID_2_dfhhb = pickle.load(open(f'./DICT_PDB_Result/{DATETIME}_DICT_PDBID_2_dfhhb.pickle', 'rb'))
else:
    for iii,fname in tqdm(enumerate(sorted(os.listdir(DIR_LIGPLOTTMP)))):
        pdbid = fname.replace('pdb','').upper()
        
        if 'hbadd_hbplus_result' in fname:
            continue
        if pdbid=='5JZI' or pdbid=='4QRR' or pdbid=='5JHD' or pdbid=='2OL3':
            print(f'{pdbid} has only delta-chain and beta-chain' )
            continue
            
        
        df_hhb = get_df(pdbid)
        alphaname, betaname, peptidename = DICT_PDBID_2_CHAINNAMES[pdbid]
        alphaname_list, betaname_list, peptidename_list = alphaname.split(', '), betaname.split(', '), peptidename.split(', ')
        
        assert len(betaname_list)!=0

        chain2abp = {} 
        for a in alphaname_list: chain2abp[a] = 'alpha'
        for b in betaname_list: chain2abp[b] = 'beta'
        for p in peptidename_list: chain2abp[p] = 'peptide'

        df_hhb['abp_donor'] = df_hhb['donor_chain_identifier'].map(chain2abp)
        df_hhb['abp_acceptor'] = df_hhb['acceptor_chain_identifier'].map(chain2abp)

        df_hhb.dropna(subset=['abp_donor'], inplace=True)
        df_hhb.dropna(subset=['abp_acceptor'], inplace=True)

        df_hhb = add_colname(df_hhb, pdbid, DICT_PDBID_2_CDRS)
        df_hhb['peptide_col'] = df_hhb.loc[(df_hhb['abp_acceptor']=='peptide'), 'colname_acceptor']
        df_hhb.loc[((df_hhb['abp_donor']=='peptide') & df_hhb['peptide_col'].isna()), 'peptide_col'] = df_hhb['colname_donor']
        df_hhb['beta_col'] = df_hhb.loc[(df_hhb['abp_acceptor']=='beta'), 'colname_acceptor']
        df_hhb.loc[((df_hhb['abp_donor']=='beta') & df_hhb['beta_col'].isna()), 'beta_col'] = df_hhb['colname_donor']
        df_hhb['alpha_col'] = df_hhb.loc[(df_hhb['abp_acceptor']=='alpha'), 'colname_acceptor']
        df_hhb.loc[((df_hhb['abp_donor']=='alpha') & df_hhb['alpha_col'].isna()), 'alpha_col'] = df_hhb['colname_donor']

        DICT_PDBID_2_dfhhb[pdbid] = df_hhb

    pickle.dump(DICT_PDBID_2_dfhhb, open(f'./DICT_PDB_Result/{DATETIME}_DICT_PDBID_2_dfhhb.pickle','wb'))



# Analysis by redidues

In [9]:

def get_attention_and_hhb_relationship(strong_atten=False):
    givenPEP_distributedTCR_by_head = {}
    givenTCR_distributedPEP_by_head = {}

    for hi in tqdm(range(HEAD_COUNT)):
        givenTCR_distributedPEP_by_head[hi] = []
        givenPEP_distributedTCR_by_head[hi] = []
        for p, (a1_by_head, a2_by_head) in DICT_PDBID_2_Atten12.items():
            if p not in pospred_ids:
                continue

            a1, a2 = calc_melt_df(DICT_PDBID_2_MELTDIST[p], a1_by_head[hi], a2_by_head[hi])
            a1['pdbid'] = p
            a2['pdbid'] = p
#             print(p, a1.shape)
            if strong_atten==True:
                temp1 = a1[(a1['value_y'] > (a1['value_y'].mean() + STD_THH * a1['value_y'].std()))]
                temp2 = a2[(a2['value_y'] > (a2['value_y'].mean() + STD_THH * a2['value_y'].std()))]
            else:
                temp1 = a1[(a1['value_y'] <= (a1['value_y'].mean() + STD_THH * a1['value_y'].std()))]
                temp2 = a2[(a2['value_y'] <= (a2['value_y'].mean() + STD_THH * a2['value_y'].std()))]
            temp1['head'] = hi
            temp2['head'] = hi
            givenPEP_distributedTCR_by_head[hi] += [temp1]
            givenTCR_distributedPEP_by_head[hi] += [temp2]

    givenPEP_distributedTCR_list,givenTCR_distributedPEP_list = [], []
    for hi in range(4):
        df1_strong_atten = pd.concat(givenPEP_distributedTCR_by_head[hi])
        givenPEP_distributedTCR_list.append(df1_strong_atten)
        df2_strong_atten = pd.concat(givenTCR_distributedPEP_by_head[hi])
        givenTCR_distributedPEP_list.append(df2_strong_atten)

    return givenPEP_distributedTCR_by_head, givenTCR_distributedPEP_by_head, givenPEP_distributedTCR_list,givenTCR_distributedPEP_list


In [10]:
pospred_ids = [k for k,v in DICT_PDBID_2_model_out.items() if DICT_PDBID_2_model_out[k][-1]>0.5]


strong_givenPEP_distributedTCR_by_head, strong_givenTCR_distributedPEP_by_head, strong_givenPEP_distributedTCR_list, strong_givenTCR_distributedPEP_list =\
get_attention_and_hhb_relationship(strong_atten=True)


general_givenPEP_distributedTCR_by_head, general_givenTCR_distributedPEP_by_head, general_givenPEP_distributedTCR_list, general_givenTCR_distributedPEP_list =\
get_attention_and_hhb_relationship(strong_atten=False)



  0%|                                      | 0/4 [00:00<?, ?it/s]/var/folders/0t/4gd_k20d0_l07n5xyz8pvych0000gn/T/ipykernel_14589/2248405354.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp1['head'] = hi
/var/folders/0t/4gd_k20d0_l07n5xyz8pvych0000gn/T/ipykernel_14589/2248405354.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp2['head'] = hi
100%|██████████████████████████████| 4/4 [00:02<00:00,  1.62it/s]


In [11]:
# for pdbi, pdbid in enumerate(DICT_PDBID_2_Atten12.keys()):
def get_statistics_result(general_givenPEP_distributedTCR_by_head):
    statistics_result = []
    for pdbid in pd.concat(general_givenPEP_distributedTCR_by_head[0]).pdbid.unique():
        for hi in range(4):
            df_datalist = pd.concat(general_givenPEP_distributedTCR_by_head[hi]) #[pdbi]
            temp = df_datalist[(df_datalist.pdbid == pdbid)]
            df_hhb = DICT_PDBID_2_dfhhb[pdbid].dropna(subset=['colname_donor']).dropna(subset=['colname_acceptor'])
            bcount = temp[(temp.peptide + temp.tcr).isin((df_hhb['peptide_col'] + df_hhb['beta_col']).values)]
            acount = temp[(temp.peptide + temp.tcr).isin((df_hhb['peptide_col'] + df_hhb['alpha_col']).values)]
            count_for_interaction = bcount.shape[0] + acount.shape[0]
            ratio = count_for_interaction/temp.shape[0]
            a = {'pdbid':pdbid, 'head':hi, 'hbond_count':df_hhb.shape[0], 'interaction_count':temp.shape[0], 'ratio':ratio}
            statistics_result.append(a)
    return statistics_result

# Analysis

## Which AA has Hbond?

In [12]:
all_hbonds_list = []
general_interaction_list = []

for headi in range(4):
    for iii, temp in enumerate(strong_givenPEP_distributedTCR_by_head[headi]):
        if len(temp)==0:continue
        pdbid = temp['pdbid'].unique().item()

        df_hhb = DICT_PDBID_2_dfhhb[pdbid][['peptide_col', 'beta_col', 'alpha_col']]
        df_hhb['beta_col'].fillna('', inplace=True)
        df_hhb['alpha_col'].fillna('', inplace=True)
        df_hhb = df_hhb.drop_duplicates(subset=['beta_col','alpha_col'])
        df_hhb = df_hhb[(df_hhb['beta_col']!='') | (df_hhb['alpha_col']!='')]
        df_hhb['pdbid'] = pdbid
        all_hbonds_list.append(df_hhb)
        df0 = pd.concat([
                general_givenPEP_distributedTCR_by_head[headi][iii], 
                strong_givenPEP_distributedTCR_by_head[headi][iii]
                    ])
        general_interaction_list.append(df0)

    df_all_hbonds_list = pd.concat(all_hbonds_list)
    df_general_interaction_list = pd.concat(general_interaction_list)

/Users/kyoheikoyama/.pyenv/versions/miniconda3-latest/envs/tcrpred/lib/python3.8/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [13]:
df_all_hbonds_list['is_beta'] = df_all_hbonds_list['beta_col'].apply(len)>1
df_all_hbonds_list['is_alpha'] = df_all_hbonds_list['alpha_col'].apply(len)>1

In [14]:
print('# of hhbonds b/w peptides and TCRs', df_all_hbonds_list.shape[0])

# of hhbonds b/w peptides and TCRs 276


In [15]:
df_all_hbonds_list['tcr'] = df_all_hbonds_list['beta_col'] + df_all_hbonds_list['alpha_col']
df_all_hbonds_list['tcr'] = df_all_hbonds_list['tcr'].str.strip()

df_all_hbonds_list.peptide = df_all_hbonds_list.peptide_col 
peptide_pos = df_all_hbonds_list.peptide_col.str.split('_').apply(lambda x: x[0])
tcr_pos = df_all_hbonds_list.tcr.str.split('_').apply(lambda x: x[0])

/var/folders/0t/4gd_k20d0_l07n5xyz8pvych0000gn/T/ipykernel_14589/3925579963.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_all_hbonds_list.peptide = df_all_hbonds_list.peptide_col


In [16]:
print(pd.value_counts(peptide_pos).to_dict())

print(pd.value_counts(tcr_pos).to_dict())

print()
df_general_interaction_list = df_general_interaction_list.drop_duplicates(subset=['tcr','pdbid'])

peptide_pos = df_general_interaction_list.peptide.str.split('_').apply(lambda x: x[0])
tcr_pos = df_general_interaction_list.tcr.str.split('_').apply(lambda x: x[0])

print(pd.value_counts(peptide_pos).to_dict())

print(pd.value_counts(tcr_pos).to_dict())

{'Y': 23, 'R': 23, 'G': 22, 'Q': 22, 'K': 22, 'T': 20, 'I': 19, 'P': 19, 'E': 19, 'W': 17, 'M': 17, 'S': 15, 'F': 9, 'L': 9, 'A': 8, 'N': 6, 'V': 4, 'D': 2}
{'S': 45, 'N': 38, 'G': 37, 'Y': 28, 'R': 19, 'E': 17, 'D': 16, 'Q': 15, 'L': 12, 'A': 10, 'V': 10, 'P': 10, 'T': 8, 'H': 6, 'W': 5}

{'A': 746, 'C': 111, 'F': 88, 'I': 48, 'G': 46, 'E': 24, 'D': 3, 'L': 2}
{'S': 171, 'G': 145, 'A': 139, 'L': 88, 'T': 84, 'Y': 62, 'Q': 51, 'N': 46, 'E': 46, 'V': 43, 'D': 35, 'F': 34, 'R': 29, 'K': 26, 'I': 25, 'W': 20, 'P': 14, 'M': 7, 'H': 3}


## Which AA is focused by strong attentions?

In [17]:
df_strong_attention = pd.concat(
    [pd.concat(strong_givenTCR_distributedPEP_by_head[i]) for i in range(4)])

df_strong_attention = df_strong_attention.drop_duplicates(subset=['tcr','pdbid'])

In [18]:
print(pd.value_counts(df_strong_attention['tcr'].str.split('_').apply(lambda x: x[0])).to_dict())

{'Y': 25, 'V': 18, 'D': 16, 'G': 15, 'T': 14, 'Q': 13, 'K': 12, 'L': 12, 'S': 12, 'E': 6, 'R': 5, 'F': 4, 'N': 3, 'A': 3, 'W': 3, 'M': 2, 'I': 2, 'P': 1}


In [19]:
df_strong_attention.drop_duplicates(subset=['tcr','pdbid']).drop(columns=['peptide'])

,tcr,value_x,value_y,pdbid,head
60,M_8,12.273016,0.404120,5WKF,0
74,S_17,21.841444,0.303707,3PQY,0
173,T_11,15.604029,0.284687,4MJI,0
228,V_11,17.803051,0.316055,4P2Q,0
107,G_8,18.812706,0.436168,1J8H,0
...,...,...,...,...,...
137,G_20,6.715578,0.357931,6EQB,3
182,G_22,14.311815,0.326428,6EQB,3
73,N_8,9.610468,0.294179,2VLR,3
139,G_20,5.733288,0.401790,6EQA,3


## Bond file reference


https://github.com/mmravic314/bin/blob/master/HBplus/hbplus.man

```
============================================================================
                         Table I: *.hb2 format

01-13 Donor Atom, including . . .

    01    Chain ID (defaults to '-')
    02-05 Residue Number 
    06    Insertion Code (defaults to '-')
    07-09 Amino Acid Three Letter Code
    10-13 Atom Type Four Letter Code

15-27 Acceptor Atom, same format as Donor atom
28-32 Donor - Acceptor distance, in Angstroms
34-35 Atom Categories - M(ain-chain), S(ide-chain) or H(etatm) - of D & A
37-39 Gap between donor and acceptor groups, in amino acids
      (-1 if not applicable)
41-45 Distance between the CA atoms of the donor and acceptor residues
      (-1 if one of the two atoms is in a hetatm)
47-51 Angle formed by the Donor and Acceptor at the hydrogen, in degrees.
      (-1 if the hydrogen is not defined)
53-57 Distance between the hydrogen and the Acceptor, in Angstroms
      (-1 if the hydrogen is not defined)
59-63 The smaller angle at the Acceptor formed by the hydrogen and an
      acceptor antecedent (-1 if the hydrogen, or the acceptor antecedent,
      is not defined)
65-69 The smaller angle at the Acceptor formed by the donor and an acceptor
      antecedent (-1 if not applicable)
71-75 Count of hydrogen bonds
============================================================================
```


In [20]:
df_strong_attention = pd.concat(
    [pd.concat(strong_givenTCR_distributedPEP_by_head[i]) for i in range(4)])
df_strong_attention = df_strong_attention.drop_duplicates(subset=['tcr','pdbid']).drop(columns=['peptide'])

df_normal_attention = pd.concat(
    [pd.concat(general_givenTCR_distributedPEP_by_head[i]) for i in range(4)])
df_normal_attention = df_normal_attention.drop_duplicates(subset=['tcr','pdbid'])

isin_strong = df_normal_attention[['tcr','pdbid']].apply(tuple, axis=1).isin(df_strong_attention[['tcr','pdbid']].apply(tuple, axis=1))
df_normal_attention = df_normal_attention[~isin_strong]


In [21]:
# text = "1D9KA1D9K/A0004-ARG N   A1D9K/A0023-SER O    2.90MM  19 5.00159.8 1.94138.4 144.0     1"
def read_hhb_text(text):
    ress_atoms = text[:45].replace('  ',' ').replace('  ',' ').replace('  ',' ').replace('  ',' ').split(' ')
    donor_res = ress_atoms[0]
    donor_atom = ress_atoms[1]
    acceptor_res = ress_atoms[2]
    acceptor_atom = ress_atoms[3]
    da_dist = float(text[45:49])
    donor_mainorside = text[49:50]
    acceptor_mainorside = text[50:51]
    gap = int(text[53:55])
    da_dist_inCAatoms = float(text[56:60])
    Angle_inHydrogen = float(text[60:65])
    Dist_bw_H_n_Acceptor = float(text[66:70])
    smaller_angle_in_A = float(text[70:75])
    smaller_angle_in_D = float(text[76:81])
    count = int(text[81:])
    return {
            'donor_res':donor_res, 'donor_atom':donor_atom,
            'acceptor_res':acceptor_res,'acceptor_atom':acceptor_atom,
            'da_dist':da_dist, 'donor_mainorside':donor_mainorside,
            'acceptor_mainorside':acceptor_mainorside, 'gap':gap,
            'da_dist_inCAatoms':da_dist_inCAatoms, 'Angle_inHydrogen':Angle_inHydrogen,
            'Dist_bw_H_n_Acceptor':Dist_bw_H_n_Acceptor, 'smaller_angle_in_A':smaller_angle_in_A,
            'smaller_angle_in_D':smaller_angle_in_D, 'count':count,
    }

def read_hhb_file(fname):
    df = pd.read_table(fname)
    return pd.DataFrame([read_hhb_text(row.item()) for i, row in df.iterrows()])

def res_process(text, pdbid):
    p = pdbid.upper()
    text = text.replace(p, '').split('-')
    return text #[0], text[1]


In [22]:
filelist = sorted(os.listdir("/Users/kyoheikoyama/workspace/ligplottmp/hbadd_hbplus_result/"))
filelist = [f for f in filelist if '.hhb' in f]
print(filelist)

root = '/Users/kyoheikoyama/workspace/ligplottmp/hbadd_hbplus_result/'

df_hhb_list = []
for f in filelist:
    pdbid = f.split('.hhb')[0].replace('pdb','')
    temp_df_hhb = read_hhb_file(os.path.join(root, f))
    temp_df_hhb['pdbid'] = pdbid.upper()
    df_hhb_list.append(temp_df_hhb)

df_hhb_inner = pd.concat(df_hhb_list)

['pdb1d9k.hhb', 'pdb1fyt.hhb', 'pdb1j8h.hhb', 'pdb1kj2.hhb', 'pdb1lp9.hhb', 'pdb1mi5.hhb', 'pdb1qrn.hhb', 'pdb1u3h.hhb', 'pdb1ymm.hhb', 'pdb2bnq.hhb', 'pdb2bnr.hhb', 'pdb2icw.hhb', 'pdb2j8u.hhb', 'pdb2nx5.hhb', 'pdb2ol3.hhb', 'pdb2uwe.hhb', 'pdb2vlj.hhb', 'pdb2vlk.hhb', 'pdb2vlr.hhb', 'pdb2ypl.hhb', 'pdb2z31.hhb', 'pdb3d39.hhb', 'pdb3d3v.hhb', 'pdb3kpr.hhb', 'pdb3kps.hhb', 'pdb3mbe.hhb', 'pdb3mv7.hhb', 'pdb3mv8.hhb', 'pdb3mv9.hhb', 'pdb3pl6.hhb', 'pdb3pqy.hhb', 'pdb3pwp.hhb', 'pdb3qiu.hhb', 'pdb3uts.hhb', 'pdb3utt.hhb', 'pdb3vxm.hhb', 'pdb3vxr.hhb', 'pdb3vxs.hhb', 'pdb3vxu.hhb', 'pdb3w0w.hhb', 'pdb4gg6.hhb', 'pdb4jfd.hhb', 'pdb4jfe.hhb', 'pdb4jrx.hhb', 'pdb4jry.hhb', 'pdb4mji.hhb', 'pdb4ozf.hhb', 'pdb4ozg.hhb', 'pdb4ozh.hhb', 'pdb4ozi.hhb', 'pdb4p2o.hhb', 'pdb4p2q.hhb', 'pdb4p2r.hhb', 'pdb4p4k.hhb', 'pdb4qok.hhb', 'pdb4qrr.hhb', 'pdb4z7u.hhb', 'pdb4z7v.hhb', 'pdb5brz.hhb', 'pdb5d2l.hhb', 'pdb5e9d.hhb', 'pdb5isz.hhb', 'pdb5jhd.hhb', 'pdb5jzi.hhb', 'pdb5ks9.hhb', 'pdb5men.hhb', 'pdb5nht.

In [23]:
!wc '/Users/kyoheikoyama/workspace/ligplottmp/hbadd_hbplus_result/pdb1d9k.hhb'

     966    9689   85008 /Users/kyoheikoyama/workspace/ligplottmp/hbadd_hbplus_result/pdb1d9k.hhb


In [24]:
!cat '/Users/kyoheikoyama/workspace/ligplottmp/hbadd_hbplus_result/pdb1d9k.hhb'

1D9KA1D9K/A0004-ARG N   A1D9K/A0023-SER O    2.90MM  19 5.00159.8 1.94138.4 144.0     1
1D9KA1D9K/A0023-SER N   A1D9K/A0004-ARG O    2.73MM  19 5.00115.0 2.15142.0 156.9     2
1D9KA1D9K/A0008-GLN N   A1D9K/A0110-THR OG1  2.77MS  96 3.87155.0 1.83109.0 100.3     3
1D9KA1D9K/A0010-LEU N   A1D9K/A0111-ARG O    2.74MM  95 6.16129.1 2.00150.0 159.1     4
1D9KA1D9K/A0113-ALA N   A1D9K/A0010-LEU O    2.90MM  97 6.08134.7 2.10128.5 141.6     5
1D9KA1D9K/A0012-VAL N   A1D9K/A0113-ALA O    3.11MM  95 6.63142.8 2.26139.9 149.6     6
1D9KA1D9K/A0115-SER N   A1D9K/A0012-VAL O    2.86MM  97 6.00121.3 2.21148.4 165.1     7
1D9KA1D9K/A0115-SER OG  A1D9K/A0012-VAL O    3.19SM  97 6.00168.4 2.21116.9 113.3     8
1D9KA1D9K/A0016-GLU N   A1D9K/A0013-TRP O    2.83MM   3 5.92135.0 2.03163.9 174.5     9
1D9KA1D9K/A0014-GLU N   A1D9K/A0115-SER O    2.92MM  95 5.92167.3 1.93156.7 156.7    10
1D9KA1D9K/A0015-GLY N   A1D9K/A0080-SER O    3.05MM  64 5.10112.6 2.53142.9 143.7    11
1D9KA1D9K/A0080-SER OG  A1D9K/A0

In [25]:
df_hhb_inner[df_hhb_inner.pdbid == '1D9K']

,donor_res,donor_atom,acceptor_res,acceptor_atom,da_dist,donor_mainorside,acceptor_mainorside,gap,da_dist_inCAatoms,Angle_inHydrogen,Dist_bw_H_n_Acceptor,smaller_angle_in_A,smaller_angle_in_D,count,pdbid
0,1D9KA1D9K/A0023-SER,N,A1D9K/A0004-ARG,O,2.73,M,M,19,5.00,115.0,2.15,142.0,156.9,2,1D9K
1,1D9KA1D9K/A0008-GLN,N,A1D9K/A0110-THR,OG1,2.77,M,S,96,3.87,155.0,1.83,109.0,100.3,3,1D9K
2,1D9KA1D9K/A0010-LEU,N,A1D9K/A0111-ARG,O,2.74,M,M,95,6.16,129.1,2.00,150.0,159.1,4,1D9K
3,1D9KA1D9K/A0113-ALA,N,A1D9K/A0010-LEU,O,2.90,M,M,97,6.08,134.7,2.10,128.5,141.6,5,1D9K
4,1D9KA1D9K/A0012-VAL,N,A1D9K/A0113-ALA,O,3.11,M,M,95,6.63,142.8,2.26,139.9,149.6,6,1D9K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,1D9KH1D9K/H0186-VAL,N,H1D9K/H0173-CYS,O,3.08,M,M,13,4.80,140.2,2.24,121.1,130.2,962,1D9K
961,1D9KH1D9K/H0184-ILE,N,H1D9K/H0175-VAL,O,3.08,M,M,9,5.29,121.3,2.44,152.9,152.5,963,1D9K
962,1D9KH1D9K/H0180-LEU,N,H1D9K/H0177-HIS,O,3.23,M,M,3,5.74,132.1,2.47,114.7,118.7,964,1D9K
963,1D9KH1D9K/H0179-SER,N,H1D9K/H0177-HIS,ND1,2.86,M,S,2,5.39,158.4,1.90,116.9,110.7,965,1D9K
